In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats

# fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"

In [2]:
year = 2023
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=year,
    season_type_all_star="Regular Season",
)
df2 = stats.get_data_frames()[0]
df2.insert(0,"GAME_ID",df2.pop("GAME_ID"))
df4 = df2.groupby("GAME_ID")
df4_1 = df4.nth(0)
df4_2 = df4.nth(1)
df4_1.columns = ["GAME_ID"] + [s + "1" for s in df4_1.columns if s != "GAME_ID"]
df4_2.columns = ["GAME_ID"] + [s + "2" for s in df4_2.columns if s != "GAME_ID"]
df4_3 = pd.merge(df4_1, df4_2, on="GAME_ID")
df4_4 = df4.nth(1)
df4_5 = df4.nth(0)
df4_4.columns = ["GAME_ID"] + [s + "1" for s in df4_4.columns if s != "GAME_ID"]
df4_5.columns = ["GAME_ID"] + [s + "2" for s in df4_5.columns if s != "GAME_ID"]
df4_6 = pd.merge(df4_4, df4_5, on="GAME_ID")
df5 = pd.concat([df4_3,df4_6])

In [3]:
stats = leaguedashteamstats.LeagueDashTeamStats(measure_type_detailed_defense="Advanced")
df6 = stats.get_data_frames()[0]
df6 = df6.sort_values("DEF_RATING").reset_index(drop=True)
top_def = df6["TEAM_ID"].iloc[:10].to_list()
df6 = df6.sort_values("NET_RATING").reset_index(drop=True)
top_teams = df6["TEAM_ID"].iloc[-10:].to_list()
bot_teams = df6["TEAM_ID"].iloc[:-10].to_list()

In [4]:
year = 2023
df7 = pd.read_parquet(box_DIR+f"NBA_Box_P_Base_{year}.parquet")

In [5]:
df8 = df5[df5["TEAM_ID2"].isin(top_def)]
game_ids = df8[["GAME_ID","TEAM_ID1"]]
game_ids.columns = ["GAME_ID","TEAM_ID"]
df9 = pd.merge(game_ids,df7,on=["GAME_ID","TEAM_ID"])
cols = [ 'MIN', 'FGM','FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS',]
df102 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])[cols].mean().reset_index()
df101 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])["FGM"].count().reset_index()
df101 = df101.rename(columns={"FGM":"GP"})
df10 = pd.merge(df101,df102,on=["PLAYER_ID","PLAYER_NAME"])
df10.iloc[:,2:] = df10.iloc[:,2:].round(2)
df10["FG_PCT"] = round(df10["FGM"]/df10["FGA"],3)*100
df10["FG3_PCT"] = round(df10["FG3M"]/df10["FG3A"],3)*100
df10["FT_PCT"] = round(df10["FTM"]/df10["FTA"],3)*100
df10["TS_PCT"] = round(df10["PTS"]*0.5/(df10["FGA"]+0.44*df10["FTA"]),3)*100
df10["P"]=img_DIR_P + df10["PLAYER_ID"].astype(str) + ".png"
df10["P"] = df10["P"].apply(path_to_image_html)

var = "PTS"
df10 = df10.query("GP > 4")
df11 = df10.sort_values(var,ascending=False).reset_index(drop=True)
df11.index +=1


,PLAYER_NAME,P,PTS,TS_PCT,GP
1,Luka Doncic,,36.11,64.6,19
2,Joel Embiid,,33.27,63.7,11
3,Shai Gilgeous-Alexander,,31.26,63.9,23
4,Giannis Antetokounmpo,,30.35,64.5,23
5,Devin Booker,,29.06,59.0,18
6,Nikola Jokic,,27.83,63.5,23
7,Kevin Durant,,27.81,61.6,21
8,Kyrie Irving,,27.56,60.3,18
9,Tyrese Maxey,,26.87,55.4,23
10,Jayson Tatum,,26.43,58.9,23


In [ ]:
df8 = df5[df5["TEAM_ID2"].isin(top_teams)]
game_ids = df8[["GAME_ID","TEAM_ID1"]]
game_ids.columns = ["GAME_ID","TEAM_ID"]
df9 = pd.merge(game_ids,df7,on=["GAME_ID","TEAM_ID"])
cols = [ 'MIN', 'FGM','FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS',]
df102 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])[cols].mean().reset_index()
df101 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])["FGM"].count().reset_index()
df101 = df101.rename(columns={"FGM":"GP"})
df10 = pd.merge(df101,df102,on=["PLAYER_ID","PLAYER_NAME"])
df10.iloc[:,2:] = df10.iloc[:,2:].round(2)
df10["FG_PCT"] = round(df10["FGM"]/df10["FGA"],3)*100
df10["FG3_PCT"] = round(df10["FG3M"]/df10["FG3A"],3)*100
df10["FT_PCT"] = round(df10["FTM"]/df10["FTA"],3)*100
df10["TS_PCT"] = round(df10["PTS"]*0.5/(df10["FGA"]+0.44*df10["FTA"]),3)*100
df10["P"]=img_DIR_P + df10["PLAYER_ID"].astype(str) + ".png"
df10["P"] = df10["P"].apply(path_to_image_html)

var = "PTS"
df10 = df10.query("GP > 4")
df11 = df10.sort_values(var,ascending=False).reset_index(drop=True)
df11.index +=1
df12 = df11[["PLAYER_NAME","P",var,"TS_PCT","GP"]].head(10)
HTML(df12.to_html(escape=False))

In [ ]:
df8 = df5[df5["TEAM_ID2"].isin(top_teams)]
game_ids1 = df8[["GAME_ID","TEAM_ID1"]]
game_ids1.columns = ["GAME_ID","TEAM_ID"]
df8 = df5[df5["TEAM_ID2"].isin(bot_teams)]
game_ids2 = df8[["GAME_ID","TEAM_ID1"]]
game_ids2.columns = ["GAME_ID","TEAM_ID"]
df9 = pd.merge(game_ids1,df7,on=["GAME_ID","TEAM_ID"])
cols = [ 'MIN', 'FGM','FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS',]
df102 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])[cols].mean().reset_index()
df101 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])["FGM"].count().reset_index()
df101 = df101.rename(columns={"FGM":"GP"})
df10 = pd.merge(df101,df102,on=["PLAYER_ID","PLAYER_NAME"])
df10.iloc[:,2:] = df10.iloc[:,2:].round(2)
df10["FG_PCT"] = round(df10["FGM"]/df10["FGA"],3)*100
df10["FG3_PCT"] = round(df10["FG3M"]/df10["FG3A"],3)*100
df10["FT_PCT"] = round(df10["FTM"]/df10["FTA"],3)*100
df10["TS_PCT"] = round(df10["PTS"]*0.5/(df10["FGA"]+0.44*df10["FTA"]),3)*100
df11 = df10.query("GP > 4")
df9 = pd.merge(game_ids2,df7,on=["GAME_ID","TEAM_ID"])
cols = [ 'MIN', 'FGM','FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS',]
df102 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])[cols].mean().reset_index()
df101 = df9.groupby(["PLAYER_ID","PLAYER_NAME"])["FGM"].count().reset_index()
df101 = df101.rename(columns={"FGM":"GP"})
df10 = pd.merge(df101,df102,on=["PLAYER_ID","PLAYER_NAME"])
df10.iloc[:,2:] = df10.iloc[:,2:].round(2)
df10["FG_PCT"] = round(df10["FGM"]/df10["FGA"],3)*100
df10["FG3_PCT"] = round(df10["FG3M"]/df10["FG3A"],3)*100
df10["FT_PCT"] = round(df10["FTM"]/df10["FTA"],3)*100
df10["TS_PCT"] = round(df10["PTS"]*0.5/(df10["FGA"]+0.44*df10["FTA"]),3)*100
df12 = df10.query("GP > 4")
df13 = pd.merge(df12,df11,on=["PLAYER_ID","PLAYER_NAME"])
df13["P"]=img_DIR_P + df13["PLAYER_ID"].astype(str) + ".png"
# df13["P"] = df13["P"].apply(path_to_image_html)

In [ ]:
df13["Diff"] = df13["PTS_x"] - df13["PTS_y"]
df14 = df13.query("PTS_x >= 25 | PTS_y >= 25")
df14 = pd.concat([df13.nsmallest(10,"Diff"),df13.nlargest(10,"Diff")])

In [ ]:
p =(
    ggplot(df14,aes(x="PTS_x",y="PTS_y",image="P"))
    + geom_point()
    + geom_abline(intercept = 0, slope = 1, color = "black",size = 2,linetype="--")
    + geom_image(size=0.175)
    + theme_idv
    + theme(
        figure_size=(6.2,5.5)
    )
    + labs(
        title="Scoring: against Top 10 Teams vs against Bot 20 Teams",
        subtitle="Min 25 PPG vs either Top 10 or Bottom 20\nPlayers above line have better PPG against Top 10 teams",
        x="Bottom 20",
        y="Top 10",
        caption="@SravanNBA | source:nba.com/stats"
    )
)
p